# Scraping mobile phone reviews from amazon.in

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
search_query="smartphones"

In [5]:
base_url="https://www.amazon.in/s?k="

In [6]:
url=base_url+search_query

In [7]:
url

'https://www.amazon.in/s?k=smartphones'

In [8]:
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87 Safari/537.36'}

In [9]:
search_response=requests.get(url,headers=header)


In [10]:
search_response.status_code
#search_response.text

200

#### A function to get the content of the page of required query

In [11]:
cookie={} # insert request cookies within{}
def getAmazonSearch(search_query,page):
    url=base_url+search_query+'&pageNumber='+str(page)
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

#### A function to get the contents of individual product pages using 'data-asin' number (unique identification number)

In [12]:
def Searchasin(asin):
    url="https://www.amazon.in/dp/"+asin
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

#### A function to pass on the link of 'see all reviews' and extract the content

In [13]:
def Searchreviews(review_link):
    url="https://www.amazon.in"+review_link
    print(url)
    page=requests.get(url,cookies=cookie,headers=header)
    if page.status_code==200:
        return page
    else:
        return "Error"

### First page product reviews extraction

In [14]:
product_names=[]
response=getAmazonSearch('smartphones',1)
soup=BeautifulSoup(response.content)

for i in soup.findAll("span",{'class':'a-size-medium a-color-base a-text-normal'}): # the tag which is common for all the names of products
    product_names.append(i.text) #adding the product names to the list

https://www.amazon.in/s?k=smartphones&pageNumber=1


findall() is used to find all the tags with the mentioned attributes and values in a page

In [15]:
product_names

['Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64GB Storage)',
 'Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery',
 'Samsung Galaxy M01 Core (Black, 2GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Redmi 8A Dual (Sea Blue, 3GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery',
 'OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128GB Storage)',
 'Samsung Galaxy M01 (Black, 3GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Samsung Galaxy M21 (Raven Black, 4GB RAM, 64GB Storage)',
 'Samsung Galaxy M31s (Mirage Black, 8GB RAM, 128GB Storage)',
 'Samsung Galaxy M31s (Mirage Black, 6GB RAM, 128GB Storage)',
 'Samsung Galaxy M31 (Space Black, 6GB RAM, 128GB Storage)',
 'Samsung Galaxy M01 Core (Black, 1GB RAM, 16GB Storage) with No Cost EMI/Additional Exchange Offers',
 'Redmi Note 8 (Neptune Blue, 4GB RAM, 64GB Storage)

In [16]:
len(product_names)

16

##### The method of extracting data-asin numbers are similar to that of product names. Only the tag details have to be changed in findall()

In [17]:
data_asin=[]
pages=8
for i in range(pages):
    response=getAmazonSearch('smartphones',i)
    soup=BeautifulSoup(response.content)
    for i in soup.findAll("div",{'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}):
        data_asin.append(i['data-asin'])
    for i in soup.findAll("div",{'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 AdHolder sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}):
        data_asin.append(i['data-asin'])

https://www.amazon.in/s?k=smartphones&pageNumber=0
https://www.amazon.in/s?k=smartphones&pageNumber=1
https://www.amazon.in/s?k=smartphones&pageNumber=2
https://www.amazon.in/s?k=smartphones&pageNumber=3
https://www.amazon.in/s?k=smartphones&pageNumber=4
https://www.amazon.in/s?k=smartphones&pageNumber=5
https://www.amazon.in/s?k=smartphones&pageNumber=6
https://www.amazon.in/s?k=smartphones&pageNumber=7


In [18]:
data_asin[:5]

['B07SDPJ4XJ', 'B089MQ622N', 'B07X4R63DF', 'B07WPVLKPW', 'B086KCCMCP']

In [19]:
len(data_asin)


128

##### By passing the data-asin numbers, we can extract the 'see all reviews' link for each product in the page

In [ ]:
links=[]
mobile_names=[]
asins=[]
for i in range(len(data_asin)):
        response=Searchasin(data_asin[i])
        soup=BeautifulSoup(response.content)
        mobile=soup.find("span",{"id":"productTitle"})
        mobile_names.append(mobile.string)
        link= soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"})
        links.append(link[0]['href'])
        asins.append(data_asin[i])
        



https://www.amazon.in/dp/B07SDPJ4XJ
https://www.amazon.in/dp/B089MQ622N
https://www.amazon.in/dp/B07X4R63DF
https://www.amazon.in/dp/B07WPVLKPW
https://www.amazon.in/dp/B086KCCMCP
https://www.amazon.in/dp/B07HGJJ559
https://www.amazon.in/dp/B089MTL51J
https://www.amazon.in/dp/B07X1KT6LD
https://www.amazon.in/dp/B089MV96RW
https://www.amazon.in/dp/B08GCSL3VV
https://www.amazon.in/dp/B07KXCKPZZ
https://www.amazon.in/dp/B089MS3GLM
https://www.amazon.in/dp/B08DJSHY94
https://www.amazon.in/dp/B07PP2K69Z
https://www.amazon.in/dp/B081FQ2X3T
https://www.amazon.in/dp/B089MQ6XLH
https://www.amazon.in/dp/B07WPVLKPW
https://www.amazon.in/dp/B07X4R63DF
https://www.amazon.in/dp/B089MQ622N
https://www.amazon.in/dp/B089MV96RW
https://www.amazon.in/dp/B089MTL51J
https://www.amazon.in/dp/B07X1KT6LD
https://www.amazon.in/dp/B086KCCMCP
https://www.amazon.in/dp/B07SDPJ4XJ
https://www.amazon.in/dp/B089MS3GLM
https://www.amazon.in/dp/B07KXCKPZZ
https://www.amazon.in/dp/B07X1KSLC3
https://www.amazon.in/dp/B08

In [ ]:
print(len(links),len(mobile_names),len(asins))
mobile_names[80]



##### Now we have the 'see all review' links. Using this link along with a page number, we can extract the reviews in any number of pages for all the products

In [ ]:
reviews=[]
for j in range(len(asins)):
    for k in range(10):
        response=Searchreviews(links[j]+'&pageNumber='+str(k))
        soup=BeautifulSoup(response.content)
        titles=soup.findAll("a",{'data-hook':"review-title"})
        bodies=soup.findAll("span",{'data-hook':"review-body"})
        stars=soup.findAll("i",{'data-hook':"review-star-rating"})
        for i in range(len(titles)):
            reviews.append([mobile_names[j],asins[j],titles[i].text,  bodies[i].text,  int(stars[i].text[0])])  
print('Number of reviews:',len(reviews))

In [ ]:

review_data = pd.DataFrame(reviews, columns = ['mobile_names','asin','title', 'body','star'])  
print(review_data.shape)

In [ ]:
review_data.sample(5)

# Save to CSV

In [ ]:
review_data.to_csv('raw_mobile_reviews.csv',index=False)
